* 人脸验证：例如：我们在火车站可以让机器扫描身份证并用脸对着摄像头后，就可以通过闸机了，因为系统验证了摄像头前面的人和身份证是同一个人，人脸验证是‘一对一’的验证，例如验证摄像头前面的人是否是身份证上的人，又例如验证手机摄像头前面的人是否与之前添加面部解锁时留存的照片是同一个人

* 人脸识别：人脸识别就是即使没有提供身份证，系统也能识别出摄像头前这个人是谁

* 本文使用一个已经训练好了的有名的模型-- faceNet。这个模型会将一张图片转换成一个向量，这个向量里包含了128个数值。我们通过对比两个向量来判断两张图片是否是同一个人。之前的激活值维度是（m, nh, nw, nc）的形式，这个模型的激活值的维度是（m, nc, nh, nw），它把深度信息放在了前面，维度具体怎么安排当前没有一个统一的标准

需要安装opencv，使用如下命令：
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple opencv-python
如果出现“THESE PACKAGES DO NOT MATCH THE HASHES....”的错误
忽略，再次输入命令即可

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
from fr_utils import * 
from inception_blocks_v2 import *

%matplotlib inline

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

# 将图片编码成128维的向量

## 1.1 使用卷积网络来进行编码
使用经过大量数据和时间训练的FaceNet模型，这个模型的结构属于Inception网络结构
关于FaceNet的输入和输出：
* 这个网络要求输入图片的像素是96 * 96.所以输入是（m, 3, 96, 96）
* 它的输出维度是（m, 128）
* 下面的代码创建来这个FaceNet模型并输出来它的参数数量

In [6]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

AttributeError: can't set attribute

In [5]:
print('Total Params:', FRmodel.count_params())

NameError: name 'FRmodel' is not defined

FaceNet最后一层是一个全连接层，这一层中又128个神经元，所以最后会输入128个激活值，这个128维的向量就代表来输入图片，也就是一张图片编码（encoding）成了128维的向量。然后我们就可以使用这个编码后的向量来对比两张脸部图片

<img src="images/distance_kiank.png" style="width:680px;height:250px;">
<caption><center> <u> <font color='purple'> **图 2**: <br> </u> <font color='purple'> </center></caption>
    
如何判断这个编码是否优秀呢？优秀的编码应该满足一下两个条件：
- 如果两张照片是同一个人，这两张图片的编码非常接近，那么编码是优秀的
- 如果两张图片是不同的人，这两张图片的编码相差很远，那么编码是优秀的

三元组（triplet）损失函数可以帮助我们使编码变得优化，这个损失函数会努力让同一个人的两张图片的编码更加接近，同时会让不同人的两张图片的编码更加不同

## 1.2 三元组损失
如果图像用$x$表示，那么它的编码可以用$f(x)$来表示，$f()$就是一个编码函数，这个函数是由神经网络学习得到的

<img src="images/f_x.png" style="width:380px;height:150px;">

<!--
We will also add a normalization step at the end of our model so that $\mid \mid f(x) \mid \mid_2 = 1$ (means the vector of encoding should be of norm 1).
!-->

我们之前学过三元组图像组合（A, P, N）:
* A 是指“Anchor”图像 --- 一个人的图像
* P 是指“Postive”图像 --- 这张图像里面的人与A是同一个人
* N 是指“Negative”图像 --- 这张图像里面的人与A不是同一个人

也就是说每个训练样本中都包含三张图片，所以$(A^{(i)}, P^{(i)}, N^{(i)})$就分别表示第$i$个训练样本的APN

因为要保证A的编码与P的编码要比A的编码与N的编码更接近，所以就有如下的不等式。通常我们会加一个$\alpha$超参数，它是用来控制这个不等关系的明显程度的：
$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

因此，下面就是我们的三元组损失函数:

$$\mathcal{J} = \sum^{N}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

上面的公式后面有个小加号，它代表了max()函数，例如"$[z]_+$"就等于$max(z,0)$。

解释一下上面的公式：
* 上面公式中标了（1）的那一块表示A与P的编码差异。这个值越小越好
* 上面公式中标了（2）的那一块表示A与N的编码差异。这个值越大越好
* $\alpha$是个超参数，是需要我们自己慢慢调参的。在本文中等于0.2

In [7]:
# 实现三元组损失函数
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    参数：
    y_true -- 这个参数我们暂时不用
    y_pred -- 这是一个python列表，里面包含了3个对象，分别是A，P，N的编码
    
    返回值：
    loss -- 损失值
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    #计算A与P的编码差异，就是公式中标（1）的那块
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)))
    
    #计算A与N的编码差异，就是公式中标（2）的那块
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)))
    
    #根据两组编码差异来计算损失
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.maximum(tf.reduce_mean(basic_loss), 0.0)
    
    return loss

In [8]:
#单元测试
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean =6, stddev = 0.1, seed = 1),
              tf.random_normal([3, 128], mean =1, stddev = 1, seed = 1),
              tf.random_normal([3, 128], mean =3, stddev = 4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print('loss = ' + str(loss.eval()))
    

loss = 350.02716


## 加载已经训练好了的模型
前面我们已经创建了一个FaceNet模型，但是并没有训练它。因为想要训练好它，需要很多数据和计算力，所以我们不会选择自己训练它，而是选择加载前任已经训练好的参数值就可以了。在本文中我们将直接使用这个前任已经训练好了的模型来进行人脸识别。下面的加载代码可能要花好几分钟时间。

In [9]:
FRmodel.compile(optimizer= 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

NameError: name 'FRmodel' is not defined

# 3.应用训练好的FaceNet模型

## 人脸验证
假设我们的门禁系统是一个人脸验证系统，那么想要进门，就需要出示自己的ID卡，并同时将面部对准摄像机，如果系统认为Id卡与摄像机面前的是同一个人，那么验证通过，门自动打开。

首先，我们得建立一个人脸数据库，这个数据库里面的人都是被允许进门的。数据库里不是存储着人脸图像，而是直接存储着人脸对应的编码向量。img_to_encoding(image_path, model)函数，就是利用输入的FRmodel模型将输入的图像转换成编码向量，这个函数内部其实就是执行了FRmodel模型的一次前向传播，也就是对输入图像进行了一次预测。

这个数据库其实就是一个python字典，它的key是人的名字，value是这个人对应的人脸编码。

In [ ]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
database = {}
# database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)

当门口有人刷他的id卡时，就能从id卡中读出他的名字，我们就可以从数据库中调出这个名字对应的人脸编码，然后这个编码与摄像头拍摄的人脸的编码进行对比，如果差异小于某个阈值，那么就算验证通过，门就会自动为他打开

In [3]:
#验证人脸
def verify(image_path, identify, database, model):
    """
    参数：
    image_path -- 需要被验证的人脸图片地址，也就是摄像头拍摄到的人脸图片
    identify -- 人名， 也就是扫描id后得到的人名
    database -- 有权开门的人脸数据库
    model -- 前面加载的已经训练好的FaceNet模型
    
    返回值：
    dist -- 返回image_path的人脸编码与identity指定的人名的人脸编码的差异
    door_open -- 如果返回True，就表示image_path和identity是同一人，那么验证通过，门开启
    """
    
    encoding = img_to_encoding(image_path, model)
    
    #对比identity关联的图片编码与image_path关联的图片编码的差异
    dist = np.linalg.norm(encoding-database[identity])
    
    # 差异小于0.7，就表明是同一个人
    # 当然，因为程序的目的是教学所以精度不高，实际应用中会取比0.7更小的阈值
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome home !")
        door_open = True
    else:
        print("It's not " + str(identity) + ",please go away ")
        door_open = False
    
    return dist, door_open

In [ ]:
verify('images/camera_0.jpg', 'younes', database, FRmodel)

下面我们用Younes这个人的图片来测试一下。images/camera_0.jpg中保存的就是Younes的图片，如下图所示。这个图片就相当于实际应用中摄像头拍摄到的图片。因为之前我们已经将Younes的图片加入到了数据库中。所以本次验证是通过的。
<img src="images/camera_0.jpg" style="width:100px;height:100px;">

下面再拿Benoit这个人的图片来试一下，images/camera_2.jpg保存的是Benoit的图片，如图所示，但是验证时，人名传入的是kian。所以本次验证是不通过的

<img src="images/camera_2.jpg" style="width:100px;height:100px;">

In [ ]:
verify('images/camera_2.jpg', 'kian', database, FRmodel)

## 3.2 人脸识别
上面实现的是人脸验证系统。如果我们的门禁是人脸验证系统，那么如果某人忘记带ID卡，就无法开门。所以我们将门禁升级成人脸识别系统，这样以来，再也不需要ID卡了，直接对准摄像头就能识别出此人是否再数据库中并且知道他是谁。当前火车站都还是人脸验证系统，以后升级成人脸识别系统后，我们就不需要带身份证来，我们的脸就是身份证

In [5]:
def who_is_it(image_path, database, model):
    """
    参数：
    image_path -- 需要被识别的人脸图像的地址
    database -- 有权开门的人脸数据库
    model -- 前面加载的已经训练好来的FaceNet模型
    
    returns:
    min_dist -- 最小差异。我们会一个个与数据库中的人脸编码进行对比，找到与待识别的图像差异最小的那个
    identity -- 差异最小的人脸图像关联的那个人的名字
    """
    
    encoding = image_to_encoding(image_Path, model)
    
    min_dist = 100
    
    #与数据库中的人脸一个个的进行对比
    for (name, db_enc) in database.items():
        
        #进行对比
        dist = np.linalg.norm(encoding-db_enc)
        
        #保存差异最小的那个
        if dist < min_dist:
            min_dist = sidt
            identity = name
    if min_dist > 0.7:
        print('Not in the database.')
    else:
        print("It's " + str(identity) + ", the distance is " + str(min_dist))
    
    return min_dist, identity

In [6]:
who_is_it("images/camera_0.jpg", database, FRmodel)

NameError: name 'database' is not defined

至此，已经知道如何开发人脸识别程序了，如果有一定的编程能力，完全可以开发出自己的web或者pc和手机端的人脸识别软件。当然，此代码只是为了教学，离商用级别还很远，如果有兴趣和能力，可以从下面几个方面进行优化：
    * 在数据库中保存一个人的多张照片（同一个人不同光照环境下的多张图片，不同时间段的多张照片等），也就是说，拿输入图片与数据库中同一个人多张图像进行验证对比，==这样可以提高精准度==。
    * 将图片中的人脸部分截取出来，也就是说，只对比人脸部分。这样就减少了其它部分，例如衣服，背景等像素的干扰。这样可以大大提高精准度。在商用软件中，可以看到我们对准人脸识别摄像头时，屏幕上我们人脸的部分就有一个方框跟随着，它就是在定位并截取我们的人脸部分图像